<a href="https://colab.research.google.com/github/irbaalifa/Tes/blob/main/database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(r"C:\Users\Bravy\Downloads\supermarket_sales - Sheet1.csv")
df.tail(10)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
990,886-18-2897,A,Yangon,Normal,Female,Food and beverages,56.56,5,14.1400,296.9400,3/22/2019,19:06,Credit card,282.80,4.761905,14.1400,4.5
991,602-16-6955,B,Mandalay,Normal,Female,Sports and travel,76.60,10,38.3000,804.3000,1/24/2019,18:10,Ewallet,766.00,4.761905,38.3000,6.0
992,745-74-0715,A,Yangon,Normal,Male,Electronic accessories,58.03,2,5.8030,121.8630,3/10/2019,20:46,Ewallet,116.06,4.761905,5.8030,8.8
993,690-01-6631,B,Mandalay,Normal,Male,Fashion accessories,17.49,10,8.7450,183.6450,2/22/2019,18:35,Ewallet,174.90,4.761905,8.7450,6.6
994,652-49-6720,C,Naypyitaw,Member,Female,Electronic accessories,60.95,1,3.0475,63.9975,2/18/2019,11:40,Ewallet,60.95,4.761905,3.0475,5.9
995,233-67-5758,C,Naypyitaw,Normal,Male,Health and beauty,40.35,1,2.0175,42.3675,1/29/2019,13:46,Ewallet,40.35,4.761905,2.0175,6.2
996,303-96-2227,B,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.6900,1022.4900,3/2/2019,17:16,Ewallet,973.80,4.761905,48.6900,4.4
997,727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,1.5920,33.4320,2/9/2019,13:22,Cash,31.84,4.761905,1.5920,7.7
998,347-56-2442,A,Yangon,Normal,Male,Home and lifestyle,65.82,1,3.2910,69.1110,2/22/2019,15:33,Cash,65.82,4.761905,3.2910,4.1
999,849-09-3807,A,Yangon,Member,Female,Fashion accessories,88.34,7,30.9190,649.2990,2/18/2019,13:28,Cash,618.38,4.761905,30.9190,6.6


In [ ]:
df.isnull().values.any()   # Check entri NaN

False

In [ ]:
df.info

<bound method DataFrame.info of       Invoice ID Branch       City Customer type  Gender  \
0    750-67-8428      A     Yangon        Member  Female   
1    226-31-3081      C  Naypyitaw        Normal  Female   
2    631-41-3108      A     Yangon        Normal    Male   
3    123-19-1176      A     Yangon        Member    Male   
4    373-73-7910      A     Yangon        Normal    Male   
..           ...    ...        ...           ...     ...   
995  233-67-5758      C  Naypyitaw        Normal    Male   
996  303-96-2227      B   Mandalay        Normal  Female   
997  727-02-1313      A     Yangon        Member    Male   
998  347-56-2442      A     Yangon        Normal    Male   
999  849-09-3807      A     Yangon        Member  Female   

               Product line  Unit price  Quantity   Tax 5%      Total  \
0         Health and beauty       74.69         7  26.1415   548.9715   
1    Electronic accessories       15.28         5   3.8200    80.2200   
2        Home and lifestyle 

## Pendapatan per minggu

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df.groupby(df.Date.dt.isocalendar().week)['gross income'].sum()

week
1      835.3995
2     1164.8190
3     1366.3505
4     1394.6135
5     1350.4975
6     1290.5635
7     1217.3140
8      825.1745
9     1391.4150
10    1353.2780
11    1142.3915
12    1196.2190
13     851.3335
Name: gross income, dtype: float64

## Member vs Nonmember

In [ ]:
df_normal = df[df['Customer type'] == 'Normal']
df_member = df[df['Customer type'] == 'Member']
df_normal.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,13:23,Credit card,324.31,4.761905,16.2155,7.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,10:37,Ewallet,604.17,4.761905,30.2085,5.3
5,699-14-3026,C,Naypyitaw,Normal,Male,Electronic accessories,85.39,7,29.8865,627.6165,2019-03-25,18:30,Ewallet,597.73,4.761905,29.8865,4.1
7,315-22-5665,C,Naypyitaw,Normal,Female,Home and lifestyle,73.56,10,36.7800,772.3800,2019-02-24,11:38,Ewallet,735.60,4.761905,36.7800,8.0


In [ ]:
df_member

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,13:08,Ewallet,522.83,4.761905,26.1415,9.1
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27,20:33,Ewallet,465.76,4.761905,23.2880,8.4
6,355-53-5943,A,Yangon,Member,Female,Electronic accessories,68.84,6,20.6520,433.6920,2019-02-25,14:36,Ewallet,413.04,4.761905,20.6520,5.8
8,665-32-9167,A,Yangon,Member,Female,Health and beauty,36.26,2,3.6260,76.1460,2019-01-10,17:15,Credit card,72.52,4.761905,3.6260,7.2
9,692-92-5582,B,Mandalay,Member,Female,Food and beverages,54.84,3,8.2260,172.7460,2019-02-20,13:27,Credit card,164.52,4.761905,8.2260,5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,267-62-7380,C,Naypyitaw,Member,Male,Electronic accessories,82.34,10,41.1700,864.5700,2019-03-29,19:12,Ewallet,823.40,4.761905,41.1700,4.3
989,430-53-4718,B,Mandalay,Member,Male,Health and beauty,75.37,8,30.1480,633.1080,2019-01-28,15:46,Credit card,602.96,4.761905,30.1480,8.4
994,652-49-6720,C,Naypyitaw,Member,Female,Electronic accessories,60.95,1,3.0475,63.9975,2019-02-18,11:40,Ewallet,60.95,4.761905,3.0475,5.9
997,727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,1.5920,33.4320,2019-02-09,13:22,Cash,31.84,4.761905,1.5920,7.7


In [ ]:
avg_gross_normal = df_normal['gross income'].sum()/len(df_normal.index)
avg_gross_member = df_member['gross income'].sum()/len(df_member.index)

print(f'Average Gross Income dari customer normal: {avg_gross_normal}')
print(f'Average Gross Income dari customer member: {avg_gross_member}')

Average Gross Income dari customer normal: 15.148707414829659
Average Gross Income dari customer member: 15.609109780439123


## Perbandingan keuntungan antar product line

In [ ]:
product_line = np.asarray(df[['Product line']])
np.unique(product_line)

array(['Electronic accessories', 'Fashion accessories',
       'Food and beverages', 'Health and beauty', 'Home and lifestyle',
       'Sports and travel'], dtype=object)

In [ ]:
df_EA = df[df['Product line'] == 'Electronic accessories']
df_FA = df[df['Product line'] == 'Fashion accessories']
df_FB = df[df['Product line'] == 'Food and beverages']
df_HB = df[df['Product line'] == 'Health and beauty']
df_HL = df[df['Product line'] == 'Home and lifestyle']
df_ST = df[df['Product line'] ==  'Sports and travel']

In [ ]:
gross_EA = df_EA['gross income'].sum()
gross_FA = df_FA['gross income'].sum()
gross_FB = df_FB['gross income'].sum()
gross_HB = df_HB['gross income'].sum()
gross_HL = df_HL['gross income'].sum()
gross_ST = df_ST['gross income'].sum()

print(f'Gross Income dari Electronic accessories: {gross_EA}')
print(f'Gross Income dari Fashion accessories: {gross_FA}')
print(f'Gross Income dari Food and beverages: {gross_FB}')
print(f'Gross Income dari Health and beauty: {gross_HB}')
print(f'Gross Income dari Home and lifestyle: {gross_HL}')
print(f'Gross Income dari Sports and travel: {gross_ST}')

Gross Income dari Electronic accessories: 2587.5015
Gross Income dari Fashion accessories: 2585.995
Gross Income dari Food and beverages: 2673.5640000000003
Gross Income dari Health and beauty: 2342.559
Gross Income dari Home and lifestyle: 2564.853
Gross Income dari Sports and travel: 2624.8965


## Perbandingan keuntungan cabang

In [ ]:
cabang = np.asarray(df[['Branch']])
np.unique(cabang)

array(['A', 'B', 'C'], dtype=object)

In [ ]:
df_A = df[df['Branch'] == 'A']
df_B = df[df['Branch'] == 'B']
df_C = df[df['Branch'] == 'C']

In [ ]:
gross_A = df_A['gross income'].sum()
gross_B = df_B['gross income'].sum()
gross_C = df_C['gross income'].sum()

print(f'Gross Income dari cabang Yangon: {gross_A}')
print(f'Gross Income dari cabang Mandalay: {gross_B}')
print(f'Gross Income dari cabang Naypyitaw: {gross_C}')

Gross Income dari cabang Yangon: 5057.1605
Gross Income dari cabang Mandalay: 5057.032
Gross Income dari cabang Naypyitaw: 5265.1765
